In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

In [3]:
airbnb=gpd.read_file("../inside_airbnb_data/neighbourhoods.geojson")

## explore the neighbor range

In [4]:
interactive_map = airbnb.explore()
interactive_map.save("airbnb.html")

## plot all the housing point on the map

In [5]:
import pandas as pd
import geopandas as gpd
import folium
import json

listings_df = pd.read_csv('../inside_airbnb_data/listings.csv')  # Replace with your path to the listings CSV file

with open('../inside_airbnb_data/neighbourhoods.geojson') as f:  # Replace with your path to the GeoJSON file
    neighborhoods_data = json.load(f)

center_lat = listings_df.latitude.mean()
center_lon = listings_df.longitude.mean()

# Initialize the map centered around the average latitude and longitude of the listings
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

folium.GeoJson(
    neighborhoods_data,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': '#a6cee3',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)


for idx, row in listings_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],  # 后面可以customize这个popup message
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

m.save('listings_map.html')  # Save the map to an HTML file


## average price for each neighbor

In [6]:
import matplotlib
import branca.colormap as cm
listings_df['price'] = listings_df['price'].replace('[\$,]', '', regex=True).astype(float)
average_prices = listings_df.groupby('neighbourhood')['price'].mean().reset_index()


min_price, max_price = average_prices['price'].min(), average_prices['price'].max()
norm = matplotlib.colors.Normalize(vmin=min_price, vmax=max_price, clip=True)
mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=matplotlib.cm.RdYlGn)

m = folium.Map(location=[listings_df.latitude.mean(), listings_df.longitude.mean()], zoom_start=12)
for feature in neighborhoods_data['features']:
    neighborhood_name = feature['properties']['neighbourhood']
    geom = feature['geometry']
    # Find the average price for the neighborhood
    avg_price = average_prices[average_prices['neighbourhood'] == neighborhood_name]['price'].values
    if avg_price:
        color = matplotlib.colors.rgb2hex(mapper.to_rgba(avg_price[0]))
        popup_message = f"{neighborhood_name}: ${avg_price[0]:.2f} average per night"
    else:
        color = '#ffffff'  
        popup_message = f"{neighborhood_name}: No data available"

    folium.GeoJson(geom, style_function=lambda feature, color=color: {
        'fillColor': color,
        'color': 'white',
        'weight': 2,
        'fillOpacity': 0.5
    }).add_child(folium.Popup(popup_message)).add_to(m)


legend = cm.LinearColormap(colors=['green', 'yellow', 'red'], vmin=min_price, vmax=max_price)
legend.caption = 'Average Airbnb Price per Night'
m.add_child(legend)
m.save('average_price_map.html')  # Save the map to an HTML file